# NYC OpenData: Data Set Lister
This script lists all the data sets given by a NYC OpenData URL. (The script scrapes the website successfully as of September 20, 2021.)

In [ ]:
!pip install esprima
import re
import requests
import esprima
import json
import time
from datetime import datetime, timezone
from os.path import isfile
from urllib.parse import urljoin
from bs4 import BeautifulSoup, Tag
from itertools import chain, dropwhile
from random import randint
from pprint import pprint

#@title Settings
url = "https://data.cityofnewyork.us/browse?Dataset-Information_Agency=Department+of+Education+%28DOE%29&sortBy=alpha&utf8=%E2%9C%93" #@param {type:"string"}
#@markdown When providing the `url`, note that by default, the search results 
#@markdown are ordered by “relevance” which will likely cause the order of the 
#@markdown results to change while parsing multiple search result pages. 
#@markdown Consequently, it’s possible to parse all of the pages in the set and 
#@markdown still not have *all* of the search results. Use a sort order like 
#@markdown “alphabetical” for more stable results.
cache = "cache.json" #@param {type:"string"}
use_cache = "Yes" #@param ["Yes", "No"]
#@markdown The `use_cache` setting above only affects getting search results.
#@markdown Activating the setting means the provided `url` will *not* be used if
#@markdown the cache file exists.

# A regular expression to match all whitespace except line breaks
whitespace_re = re.compile(r'[^\S\r\n]+', re.DOTALL)

# A regular expression to match “# results”
results_count_re = re.compile(r'(\d+)\s+Results|(1)\s+Result', re.IGNORECASE)

# Loads a JSON a file
def load_from_cache(path):
  with open(path, 'r') as cache_file:
    return json.load(cache_file)

# Gets the first item in a subscriptable object or None
def first(iterable):
  return next(iter(iterable), None)

# Gets the first non-null item in a subscriptable object or None
def first_nonnull(iterable):
  return first(dropwhile(lambda item: item is None, iterable))

# Removes excess whitespace inside and out
def strip(value):
  return whitespace_re.sub(' ', value.strip())

# Converts UNIX time to an ISO format string
def unix_time_to_iso(timestamp):
  return datetime.fromtimestamp(timestamp, timezone.utc).isoformat()

# Scrapes the supplied NYC OpenData *browse* url for items
def get_data_sets(url, data_sets={}):
  delay_factor = 1 # This delay factor will increase exponentially on errors.
  sleep = lambda: time.sleep(randint(2, 4) * delay_factor)
  print(f'Getting search results from {url}...')
  try:
    page_data_sets, next_url, last_url, expected_length = parse_results_page(url)
    page_data_sets.update(data_sets) # Data sets already in the collection
    data_sets = page_data_sets       # override the freshly extracted ones.
    print(f'Expecting a total of {expected_length} search results...')
    sleep()
    while next_url:
      print(f'Getting additional search results from {next_url}...')
      try:
        url = next_url
        page_data_sets, next_url, new_last_url, new_expected_length \
          = parse_results_page(next_url)
        page_data_sets.update(data_sets) # Data sets already in the collection
        data_sets = page_data_sets       # override the freshly extracted ones.
        if expected_length != new_expected_length:
          print(f'The number of search results provided by the server changed '
                f'from {expected_length} to {new_expected_length}!')
          expected_length = new_expected_length
        if next_url:
          delay_factor = max(1, delay_factor // 2)
          sleep() # Take it nice and easy; the server will be angry otherwise.
        elif last_url and last_url != url:
          print(f'last_url = {last_url}, url = {url}')
          print('The server returned unexpected results. Could not extract a '
                'link to the next search results page. The returned results may'
                ' not be complete.')
      except Exception as e:
        print(f'An error occured while getting additional search results: {e}')
        print('The returned results may not be complete.')
        delay_factor = min(1800, delay_factor * 2)
        sleep()
    print(f'Finished extracting {len(data_sets)} search results.')
    if len(data_sets) != expected_length:
      print(f'The number of search results extracted did not match the expected'
            f' number of results ({expected_length}).')
    return data_sets
  except Exception as e:
    print(f'An error occured while getting the search results: {e}')

# Loads and parses the results page, returning the items and the next page’s url
def parse_results_page(url):
  make_full_url = lambda relative_url: urljoin(url, relative_url) \
                                       if relative_url \
                                       else None
  response = requests.get(url)
  response.raise_for_status() # Raises an error if the request is not successful
  soup = BeautifulSoup(response.text) # Parses the raw HTML into a structure
  data_sets = {result_element.get('data-view-id'):
                   element_to_dict(result_element)
               for result_element
               in soup.select('.browse2-result')}
  next_url = make_full_url(extract_next_url(soup))
  last_url = make_full_url(extract_last_url(soup))
  expected_length = extract_expected_length(soup)
  return (data_sets, next_url, last_url, expected_length)

# Extracts information about each result into a dictionary
def element_to_dict(element):
  def get_element(selector):
    return first(element.select(selector))
  def get_link(selector):
    element = get_element(selector)
    if element:
      return element.get('href')
  def get_text(selector):
    element = get_element(selector)
    if element:
      return strip(element.text)
  timestamp = get_element('.browse2-result-timestamp-value > '
                        + '[data-rawdatetime]') \
             .get('data-rawdatetime')
  return {'name': get_text('.browse2-result-name-link'),
          'link': get_link('.browse2-result-name-link'),
          'category': get_text('.browse2-result-category'),
          'type': get_text('.browse2-result-type-name'),
          'description': get_text('.browse2-result-description'),
          'tags': list(map(Tag.get_text, element.select('.browse2-result-topic'))),
          'updated': unix_time_to_iso(int(timestamp)),
          'apiDocLink': get_link('.browse2-result-api-link')}

# Adds details to each item by modifying its dictionary in-place
def get_details(data_sets):
  delay_factor = 1 # This delay factor will increase exponentially on errors.
  sleep = lambda: time.sleep(randint(2, 4) * delay_factor)
  for id in data_sets:
    data_set = data_sets[id]
    if 'dataDownloads' in data_set or \
       'attachments' in data_set or \
       'columns' in data_set:
       continue # Skip items with any of those keys already.
    try:
      data_set_name = data_set['name']
      details_url = data_set['link']
      print(f'Getting details for {data_set_name} from {details_url}...')
      sleep() # Take it nice and easy; the server will be angry otherwise.
      data_set_information, initial_state = parse_details_page(details_url)
      data_sets[id]['rawMetadata'] = {'dataSet': data_set_information,
                                      'initialState': initial_state}
      data_downloads = extract_data_downloads(url, data_set_information)
      attachments = extract_attachments(url, initial_state)
      columns = extract_column_schema(initial_state)
      data_sets[id]['attachments'] = attachments
      data_sets[id]['columns'] = columns
      data_sets[id]['dataDownloads'] = data_downloads
      delay_factor = max(1, delay_factor // 2)
    except ValueError as e:
      print(f'\tAn error occured while getting the details: {e}')
      delay_factor = min(300, delay_factor * 2)

# Extracts the data downloads information associated with the item
def extract_data_downloads(url_base, data_set_information):
  if data_set_information and data_set_information['distribution']:
    return [{'contentUrl': data_download['contentUrl'],
             'encodingFormat': data_download['encodingFormat']}
            for data_download
            in data_set_information['distribution']
            if data_download['@type'] == 'DataDownload']

# Extracts the attachments information associated with the item
def extract_attachments(url_base, initial_state):
  if initial_state and \
     initial_state['view'] and \
     initial_state['view']['attachments']:
    return {attachment['name']: urljoin(url_base, attachment['href'])
            for attachment
            in initial_state['view']['attachments']}

# Extracts the column schema information associated with the item
def extract_column_schema(initial_state):
  if initial_state and \
     initial_state['view'] and \
     initial_state['view']['columns']:
    return [{'name': column['fieldName'],
             'type': column['dataTypeName'],
             'humanName': column['name']}
            for column
            in sorted(initial_state['view']['columns'],
                      key=lambda _: int(_['position']))]

# Digs into the item’s page to extract additional details
def parse_details_page(url):
  response = requests.get(url)
  response.raise_for_status()
  soup = BeautifulSoup(response.text)
  data_set_information = None # The information about the data set embedded in the page
  json_objects = extract_inline_json(soup)
  for json_object in json_objects:
    if json_object and json_object['@type'] == 'Dataset': # Found the information!
      data_set_information = json_object
      break
  initial_state = None # The embedded information that the page uses to initialize its tables
  scripts = extract_inline_javascript(soup)
  for script in scripts:
    try:
      ast = esprima.parseScript(script, {'range': True}) # Guards against not-JavaScript
      if ast.type == 'Program' and \
         ast.sourceType == 'script' and \
         len(ast.body) == 1 and \
         ast.body[0].type == 'VariableDeclaration' and \
         len(ast.body[0].declarations) == 1: # Found the script with a single variable declaration
        declaration = ast.body[0].declarations[0]
        if declaration.type == 'VariableDeclarator' and \
           declaration.id.type == 'Identifier' and \
           declaration.id.name == 'initialState' and \
           declaration.init.type == 'ObjectExpression': # Found the initial state!
          json_start, json_end = declaration.init.range
          initial_state = json.loads(script[json_start:json_end])
          break
    except Exception as e:
      pass # Ignore the “script” if parsing it throws an exception.
  return (data_set_information, initial_state)

# Loads all of the inline JSON found in the page’s script tags
def extract_inline_json(soup):
  return [json.loads(element.string)
          for element
          in soup.select('script[type="application/ld+json"]')
          if element.string]

# Loads all of the inline JavaScript found in the page’s script tags
def extract_inline_javascript(soup):
  return [element.string
          for element
          in soup.select('script')
          if element.string]

# Gets the URL of the next results page
def extract_next_url(soup):
  element = soup.select('a.nextLink')
  if element:
    return element[0].get('href')

# Gets the URL of the last results page
def extract_last_url(soup):
  element = soup.select('a.lastLink')
  if element:
    return element[0].get('href')

# Gets the number of search results to expect
def extract_expected_length(soup):
  element = first(soup.select('.browse2-mobile-filter-result-count'))
  return int(first_nonnull(first(map(re.Match.groups,
                                     filter(None,
                                            map(results_count_re.match,
                                                element.stripped_strings))))))

## Getting the Search Results
The code below extracts the data from the first search results page and all subsequent pages until it cannot find a link to the next page. For each page, it looks for the search result elements and maps each one to a dictionary. The dictionary schema is as follows:
* **name** (string): the name of the item
* **link** (string): the link to the page with more information about the item
* **category** (string): the category of the item (e.g., *Education*)
* **type** (string): the type of the item (e.g., *Dataset*)
* **description** (string): a description of the item
* **tags** (set of strings): a set of tags associated with the item
* **updated** (integer): the timestamp which this item was last updated
* **apiDocLink** (string): a link to the API documentation (which might possibly be used to extract more metadata about the item)

In [ ]:
data_sets = load_from_cache(cache) if use_cache == 'Yes' and isfile(cache) else get_data_sets(url)

## Getting the Data Set Details
The code below extracts additional information about the items using the links to the items’ pages. It adds the following keys if the information is available:
* **attachments** (dict of strings): key-value pairs of file names and their corresponding links to download them
* **columns** (list of dicts): an ordered list of dicts representing column metadata
* **dataDownloads** (list of dicts of strings): a list of key-value pairs where the key is the file name and the value is the link to its URL

In the case where download of data set details is interrupted, the code below will attempt to resume progress. Simply, it checks each dictionary entry for the existence of the additional keys. If those don’t exist, it tries to retrieve them again and amends the dictionary.

In [ ]:
get_details(data_sets)

## Caching Data
The code below will save a copy of the data to storage for loading and processing later. The file name is defined in the settings above.

In [ ]:
with open(cache, 'w') as cache_file:
  json.dump(data_sets, cache_file, indent=2, sort_keys=True)